In [1]:
# -*- coding: utf-8 -*-
from flask import Flask, flash
import flask
from flask import render_template
from flask import request
import sqlite3
import numpy as np
import pandas as pd
import json
import operator
import string
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from __future__ import division
from collections import Counter
from itertools import chain

In [2]:
def getrandompatent(findrows):
    
    patent = findrows.execute("SELECT * FROM MostCitedPatents2 ORDER BY RANDOM() LIMIT 1").fetchall()[0][0]
    patent_detail = findrows.execute("SELECT * FROM Patents WHERE patent_number = ?",(patent,)).fetchall()
    
    if len(patent_detail)  == 0:
        error = 'No data for this patent'
        patents_info_dict = {}
    else:
        patents_info_dict = {}
        patents_info_dict['Patent_Number'] = patent_detail[0][0]
        patents_info_dict['Patent_Date'] = patent_detail[0][1]
        patents_info_dict['Patent_Orginazation'] = patent_detail[0][12]
        patents_info_dict['Patent_State'] = patent_detail[0][13]
        patents_info_dict['Patent_Title'] = patent_detail[0][4]
        patents_info_dict['Patent_Abstract'] = patent_detail[0][5]
        patents_info_dict['Patent_Inventor'] = patent_detail[0][6]
        patents_info_dict['Patent_Coinventor'] = patent_detail[0][7]
        error = None
    
    patents_info_dict_final = []
    patents_info_dict_final.append(patents_info_dict)
    
    return patent, patents_info_dict_final, error


def getpatentdetails(findrows, patent):
    patent_detail = findrows.execute("SELECT * FROM Patents WHERE patent_number = ?",(patent,)).fetchall()
    
    patents_info_dict_final = []
    if len(patent_detail)  == 0:
        error = 'No data for this patent'
        patents_info_dict_final = []
    else:
        patents_info_dict = {}
        patents_info_dict['Patent_Number'] = patent_detail[0][0]
        patents_info_dict['Patent_Date'] = patent_detail[0][1]
        patents_info_dict['Patent_Orginazation'] = patent_detail[0][12]
        patents_info_dict['Patent_State'] = patent_detail[0][13]
        patents_info_dict['Patent_Title'] = patent_detail[0][4]
        patents_info_dict['Patent_Abstract'] = patent_detail[0][5]
        patents_info_dict['Patent_Inventor'] = patent_detail[0][6]
        patents_info_dict['Patent_Coinventor'] = patent_detail[0][7]
        patents_info_dict_final.append(patents_info_dict)
        error = None   
    
    return patents_info_dict_final,error


def getsimilarpatents(patent,findrows):
    
    similarpatentseg = findrows.execute("SELECT similarpatents FROM SimilarPatents WHERE patent_number = ? LIMIT 1",(patent,)).fetchall()
    patents_list = []
    
    if len(similarpatentseg) == 0:
        error = 'Not computed for this patent'
        patents_list =[]
    else:
        for item in similarpatentseg[0][0].split(','):
            patent_details = findrows.execute("SELECT * FROM Patents WHERE patent_number = ?",(item,)).fetchall()
            patents = {}
            patents['Patent_Number'] = patent_details[0][0]
            patents['Patent_Date'] = patent_details[0][1]
            patents['Patent_Orginazation'] = patent_details[0][12]
            patents['Patent_State'] = patent_details[0][13]
            patents['Patent_Title'] = patent_details[0][4]
            patents['Patent_Abstract'] = patent_details[0][5]
            patents['Patent_Inventor'] = patent_details[0][6]
            patents['Patent_Coinventor'] = patent_details[0][7]
            patents_list.append(patents)
        error = None
        
    return patents_list, error

def top10patentsstats(patents_info_dict, patents_list,error):
    
    if error == None:
        
        patent_org = patents_info_dict[0]['Patent_Orginazation']
        patent_inventor = patents_info_dict[0]['Patent_Inventor']
        patent_number_img = patents_info_dict[0]['Patent_Number']
    
        similar_org =  []
        similar_first_inventor = []
        similar_inventor = []
        similar_inventor_b = []

        for item in patents_list:
            similar_inventor_a = []
            similar_org.append(item['Patent_Orginazation'])
            similar_first_inventor.append(item['Patent_Inventor'])
            similar_inventor_a.append(item['Patent_Inventor'])
            similar_inventor_a.append(item['Patent_Coinventor'])
            similar_inventor_b.append(similar_inventor_a)

        for item in similar_inventor_b:
            for subitem in item:
                similar_inventor.append(subitem.split(','))
        similar_inventor = list(chain.from_iterable(similar_inventor))

        for item in similar_inventor:
            if len(item) <= 4:
                similar_inventor.remove(item)

        count = 0
        for item in similar_inventor:
            if item == patent_inventor:
                count = count+1

        lensimilarinventor = len(similar_inventor)

        similar_org = dict(Counter(similar_org))
        similar_first_inventor = dict(Counter(similar_first_inventor))

        ax = plt.figure()
        ax = sns.barplot(similar_org.keys(),similar_org.values(),palette="BuGn_r")
        ax.grid(False)
        ax.set_xticklabels(similar_org.keys(),rotation=80)
        ax.set(xlabel="Orginization Name",ylabel="Frequency in Top10 Similar Patents")
        plt.tight_layout()
        ax.figure.savefig("static/similarorg{}.png".format(patent_number_img), dpi=80)
        ax.clear()

        bx =plt.figure()
        bx = sns.barplot(similar_first_inventor.keys(),similar_first_inventor.values(),palette="BuPu")
        bx.grid(False)
        bx.set_xticklabels(similar_first_inventor.keys(),rotation=80)
        bx.set(xlabel="Inventor Name",ylabel="Frequency in Top10 Similar Patents")
        plt.tight_layout()
        bx.figure.savefig("static/similarinventor{}.png".format(patent_number_img), dpi=80)
        bx.clear()
        
        patent_number_imgf = {'Patent_number':patent_number_img}
    
    else:
        error = 'Not computed for this patent'
        count = None
        lensimilarinventor = None
        patent_number_imgf = None
        
    return count, lensimilarinventor, error, patent_number_imgf
    

In [3]:
app = Flask(__name__)
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0

conn = sqlite3.connect('Patent-Project-Rev1.sqlite')
findrows = conn.cursor()

@app.route('/')
def my_form():
    return render_template("index8button.html")

@app.route('/', methods=['POST'])
def my_form_post():

    text = request.form['text']
    
    if text.lower() == "random":
        patent, patents_info_dict,error1 = getrandompatent(findrows)
        patents_list, error = getsimilarpatents(patent,findrows)
        count, lensimilarinventor, error2, patent_number_img = top10patentsstats(patents_info_dict, patents_list,error)
        #persimilarinventor = (count./lensimilarinventor)*100
        
    else:
        patent = text #.encode("utf-8")
        patents_info_dict,error1 = getpatentdetails(findrows, patent)
        patents_list, error = getsimilarpatents(patent,findrows)
        count, lensimilarinventor, error2, patent_number_img = top10patentsstats(patents_info_dict, patents_list,error)
        #persimilarinventor = (count./lensimilarinventor)*100
            
    return render_template("index8buttone.html",patent = patents_info_dict,
                           posts = patents_list, error1=error1, error=error, error2=error2,
                          patent_number_img=patent_number_img)

if __name__ == '__main__':
    app.run()

UnsupportedOperation: not writable